# Bert Classifier for First ICU Prediction

## Import & Inits

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

import numpy as np
np.set_printoptions(precision=2)

import pandas as pd
import pickle
import torch
from pathlib import Path

In [ ]:
from pytorch_pretrained_bert import BertTokenizer

In [ ]:
from bert_classifier.data_processor import read_df, convert_examples_to_features
from utils.splits import set_two_splits
from args import args
vars(args)

In [ ]:
ori_df = pd.read_csv(args.dataset_csv, usecols=args.cols)

In [ ]:
df = set_two_splits(ori_df.copy(), 'val')
train_ex = read_df(df.loc[(df['split'] == 'train')], 'note', 'class_label')
val_ex = read_df(df.loc[(df['split'] == 'train')], 'note', 'class_label', set_type='val')

In [ ]:
tokenizer = BertTokenizer.from_pretrained(args.bert_dir, do_lower_case=args.do_lower_case)
train_feats = convert_examples_to_features(train_ex, args.labels, args.max_seq_len, tokenizer)

In [ ]:
val_feats = convert_examples_to_features(val_ex, args.labels, args.max_seq_len, tokenizer)